In [1]:
import os
import json
import argparse
import numpy as np
from types import SimpleNamespace

import torch
import torchvision
from sklearn import preprocessing
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter

from simsiam.models import ResNet, LinearClassifier
from simsiam.transforms import load_transforms, augment_transforms2


c:\Users\user\anaconda3\envs\fstor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('./configs/train.json', 'r') as f:
    cfg= json.load(f)

In [3]:
print(cfg['model']['backbone'])
print(cfg['model']['freeze'])

resnet50
True


In [4]:

batch_size = 256
model = ResNet(
        backbone=cfg['model']['backbone'],
        num_classes=cfg['data']['num_classes'],
        pretrained=False,
        freeze=cfg['model']['freeze']
    )
model=model.encoder
model.load_state_dict(torch.load("repretrain/pretrained/pretrained.pt"))
#"runs/May17_09-30-21_DESKTOP-CVQR3S4/pretrained.pt"
#"pretrain1/pretrained.pt"

c:\Users\user\anaconda3\envs\fstor\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\fstor\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [5]:
model

Encoder(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 25

In [6]:
train_dataset = torchvision.datasets.STL10('D:/kaihsiang/結構化機器學習/byol/data', split='train', download=True,
                               transform=load_transforms(input_shape=cfg['data']['input_shape']))

test_dataset = torchvision.datasets.STL10('D:/kaihsiang/結構化機器學習/byol/data', split='test', download=True,
                               transform=load_transforms(input_shape=cfg['data']['input_shape']))

Files already downloaded and verified
Files already downloaded and verified


In [7]:
print("Input shape:", train_dataset[0][0].shape)


Input shape: torch.Size([3, 96, 96])


In [8]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=cfg['train']['batch_size'],
    num_workers=0,
    drop_last=False,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=cfg['train']['batch_size'],
    num_workers=0,
    drop_last=False,
    shuffle=True)

In [9]:
device = 'cpu'
model.to(device)

Encoder(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 25

In [10]:
LC=LinearClassifier(model.emb_dim, 10)
LC=LC.to(device)

In [11]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

        
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [12]:
model.eval()
x_train, y_train = get_features_from_encoder(model, train_loader)
x_test, y_test = get_features_from_encoder(model, test_loader)
if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

Training data shape: torch.Size([5000, 2048]) torch.Size([5000])
Testing data shape: torch.Size([8000, 2048]) torch.Size([8000])


In [13]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=256, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=256, shuffle=False)
    return train_loader, test_loader

In [14]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [15]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [16]:
writer = SummaryWriter('./repretrain/train/freeze/614')

In [17]:
optimizer = torch.optim.Adam(LC.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10
total_loss=0
for epoch in range(100):
#     train_acc = []
    total_loss=0
    num=0
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = LC(x)
        predictions = torch.argmax(logits, dim=1)
        
        num+=1
        loss = criterion(logits, y)
        total_loss += loss
        loss.backward()
        optimizer.step()
    writer.add_scalar('tune_loss', total_loss.item()/num, epoch)
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct_top1 = 0
        correct_top5 = 0
        all_predictions = []
        all_labels = []
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)

            logits = LC(x)
            _, predictions = logits.topk(5, dim=1, largest=True, sorted=True)
            all_predictions.extend(predictions[:, 0].cpu().numpy())
            all_labels.extend(y.cpu().numpy())

            correct_top1 += (predictions[:, 0] == y).sum().item()
            correct_top5 += (predictions == y.unsqueeze(1)).sum().item()
            total += y.size(0)
            
            
        f1 = f1_score(all_labels, all_predictions, average='weighted')
        top1_acc = 100 * correct_top1 / total
        top5_acc = 100 * correct_top5 / total
        print(f"Testing:Top1 accuracy: {np.mean(top1_acc)}; Top5 accuracy: {np.mean(top5_acc)}; f1-score: {np.mean(f1)}")

Testing:Top1 accuracy: 19.35; Top5 accuracy: 64.075; f1-score: 0.13719629920124018
Testing:Top1 accuracy: 25.7625; Top5 accuracy: 76.05; f1-score: 0.20518425275123073
Testing:Top1 accuracy: 27.25; Top5 accuracy: 76.4875; f1-score: 0.2479140625285695
Testing:Top1 accuracy: 27.0375; Top5 accuracy: 78.1125; f1-score: 0.23495510574757517
Testing:Top1 accuracy: 28.9125; Top5 accuracy: 77.9875; f1-score: 0.2692086792358009
Testing:Top1 accuracy: 28.925; Top5 accuracy: 79.0875; f1-score: 0.26750798003328136
Testing:Top1 accuracy: 29.15; Top5 accuracy: 78.9375; f1-score: 0.2718137621226322
Testing:Top1 accuracy: 29.3; Top5 accuracy: 79.25; f1-score: 0.27023086878774927
Testing:Top1 accuracy: 30.4375; Top5 accuracy: 80.05; f1-score: 0.2913427238521777
Testing:Top1 accuracy: 29.05; Top5 accuracy: 79.5; f1-score: 0.2701842760971597


In [54]:
total = 0

correct = 0
for x, y in train_loader:
    x = x.to(device)
    y = y.to(device)

    logits = LC(x)
    predictions = torch.argmax(logits, dim=1)
    
    total += y.size(0)
    correct += (predictions == y).sum().item()
    
acc = 100 * correct / total
print(f"Testing accuracy: {np.mean(acc)}")

Testing accuracy: 67.66


In [4]:
model2 = ResNet(
        backbone=cfg['model']['backbone'],
        num_classes=cfg['data']['num_classes'],
        pretrained=False,
        freeze=cfg['model']['freeze']
    )
model2.encoder.load_state_dict(torch.load("repretrain/pretrained/pretrained.pt"))

c:\Users\user\anaconda3\envs\fstor\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\fstor\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [5]:
device2 = 'cuda'
model2.to(device2)

ResNet(
  (encoder): Encoder(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (do

In [10]:
writer2 = SummaryWriter('repretrain/train/no freeze/')

In [11]:
optimizer = torch.optim.SGD(model2.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 10
total_loss=0
for epoch in range(100):
#     train_acc = []
    total_loss=0
    num=0
    for x, y in train_loader:

        x = x.to(device2)
        y = y.to(device2)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
        logits = model2(x)
        
        num+=1
        loss = criterion(logits, y)
        total_loss += loss
        loss.backward()
        optimizer.step()
    writer2.add_scalar('tune_loss', total_loss.item()/num, epoch)
    print(f"total loss:{total_loss}")
    
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x1, y1 in test_loader:
            x1 = x1.to(device2)
            y1 = y1.to(device2)

            logits1 = model2(x1)
            predictions = torch.argmax(logits1, dim=1)
            
            total += y1.size(0)
            correct += (predictions == y1).sum().item()
            
        acc = 100 * correct / total
        print(f"Testing accuracy: {np.mean(acc)}")
torch.save(model2.state_dict(), os.path.join(writer2.log_dir, cfg.model.name + ".pt"))

c:\Users\user\anaconda3\envs\fstor\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


total loss:92.1593246459961
Testing accuracy: 10.7875
total loss:92.1595687866211
total loss:92.1520767211914
total loss:92.14521789550781
total loss:92.15034484863281
total loss:92.14285278320312
total loss:92.14266967773438
total loss:92.12336730957031
total loss:92.13380432128906
total loss:92.13880920410156
total loss:92.13433837890625
Testing accuracy: 13.175
total loss:92.12774658203125
total loss:92.11344146728516
total loss:92.1196060180664
total loss:92.11326599121094
total loss:92.10082244873047
total loss:92.1081771850586
total loss:92.10084533691406
total loss:92.09521484375
total loss:92.09929656982422
total loss:92.09026336669922
Testing accuracy: 13.15
total loss:92.08744812011719
total loss:92.09611511230469
total loss:92.07118225097656
total loss:92.07521057128906
total loss:92.0721435546875
total loss:92.0682144165039
total loss:92.06845092773438
total loss:92.06196594238281
total loss:92.06005096435547
total loss:92.05884552001953
Testing accuracy: 13.2375
total loss

Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\fstor\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\user\AppData\Local\Temp\ipykernel_12340\1230425992.py", line 42, in <module>
    torch.save(model2.state_dict(), os.path.join(writer2.log_dir, cfg.model.name + ".pt"))
AttributeError: 'dict' object has no attribute 'model'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\fstor\lib\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\user\anaconda3\envs\fstor\lib\site-packages\IPython\core\ultratb.py", line 1310, in structured_traceback
    return FormattedTB.structured_traceback(
  File "c:\Users\user\anaconda3\envs\fstor\lib\site-packages\IPython\core\ultratb.py", line 1199, in stru